In [146]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import psycopg2 as ps
from psycopg2 import sql, connect
from getpass import getpass

In [178]:
#Connecting to AWS
import getpass
engine = ps.connect(
    database="postgres",
    user= input("Username: "),
    password= getpass.getpass("Password: "),
    host="highered-db.cx2ll8zuuepz.us-east-2.rds.amazonaws.com",
    port='5432'
)

In [148]:
cr = engine.cursor()
cr.execute('SELECT * FROM highered_data_table;')
tmp = cr.fetchall()

In [149]:
#Converting to Pandas DataFrame
dirty_df = pd.DataFrame(tmp, columns= ['Year','UNITID','FACSTAT','ARANK','HRTOTLT','HRTOTLM','HRTOTLW','INSTNM','CITY','STABBR','ZIP',
                                    'CONTROL','HLOFFER','INSTSIZE','LONGITUD','LATITUDE'])
dirty_df

,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,0,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,10,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,10,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,10,2,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,10,3,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192599,2020,232618,0,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192600,2020,232618,10,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192601,2020,232618,10,6,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192602,2020,232618,40,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100


In [150]:
# # Load the higher_ed dataset.
# dirty_df = pd.read_csv('highered_merged_data.csv')
# dirty_df

In [151]:
dirty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192604 entries, 0 to 192603
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Year      192604 non-null  int64  
 1   UNITID    192604 non-null  int64  
 2   FACSTAT   192604 non-null  int64  
 3   ARANK     192604 non-null  int64  
 4   HRTOTLT   192604 non-null  int64  
 5   HRTOTLM   192604 non-null  int64  
 6   HRTOTLW   192604 non-null  int64  
 7   INSTNM    192604 non-null  object 
 8   CITY      192604 non-null  object 
 9   STABBR    192604 non-null  object 
 10  ZIP       192604 non-null  object 
 11  CONTROL   192604 non-null  int64  
 12  HLOFFER   192604 non-null  int64  
 13  INSTSIZE  192604 non-null  int64  
 14  LONGITUD  192604 non-null  float64
 15  LATITUDE  192604 non-null  float64
dtypes: float64(2), int64(10), object(4)
memory usage: 23.5+ MB


In [152]:
dirty_df = dirty_df.drop(['FACSTAT','ARANK'], axis=1)

In [153]:
#Testing out summing values on the totals, total men and total women columns
dirty_test = dirty_df.groupby(['UNITID'], as_index=False).agg({'Year':'first','HRTOTLT': 'sum', 'HRTOTLM': 'sum', 'HRTOTLW': 'sum','INSTNM': 'first',
                                                                'CITY': 'first','STABBR': 'first','ZIP': 'first','HLOFFER': 'first',
                                                                'INSTSIZE': 'first', 'LONGITUD': 'first', 'LATITUDE': 'first'})
dirty_test.head(10)

,UNITID,Year,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,100654,2019,4036,2085,1951,Alabama A & M University,Normal,AL,35762,9,3,-86.568502,34.783368
1,100663,2019,45196,25012,20184,University of Alabama at Birmingham,Birmingham,AL,35294-0110,9,5,-86.799345,33.505697
2,100690,2019,693,553,140,Amridge University,Montgomery,AL,36117-3553,9,1,-86.174010,32.362609
3,100706,2019,6833,3288,3545,University of Alabama in Huntsville,Huntsville,AL,35899,9,3,-86.640449,34.724557
4,100724,2019,3568,1798,1770,Alabama State University,Montgomery,AL,36104-0271,9,2,-86.295677,32.364317
5,100751,2019,26208,13731,12477,The University of Alabama,Tuscaloosa,AL,35487-0100,9,5,-87.545978,33.211875
6,100760,2019,633,300,333,Central Alabama Community College,Alexander City,AL,35010,3,2,-85.945266,32.924780
7,100812,2019,1292,605,687,Athens State University,Athens,AL,35611,7,2,-86.964698,34.806793
8,100830,2019,3602,1791,1811,Auburn University at Montgomery,Montgomery,AL,36117-3596,9,3,-86.177544,32.367360
9,100858,2019,23652,13523,10129,Auburn University,Auburn,AL,36849,9,5,-85.488258,32.599378


In [154]:
#Dropping the nulls
dirty_test = dirty_test.dropna()
dirty_test

,UNITID,Year,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,100654,2019,4036,2085,1951,Alabama A & M University,Normal,AL,35762,9,3,-86.568502,34.783368
1,100663,2019,45196,25012,20184,University of Alabama at Birmingham,Birmingham,AL,35294-0110,9,5,-86.799345,33.505697
2,100690,2019,693,553,140,Amridge University,Montgomery,AL,36117-3553,9,1,-86.174010,32.362609
3,100706,2019,6833,3288,3545,University of Alabama in Huntsville,Huntsville,AL,35899,9,3,-86.640449,34.724557
4,100724,2019,3568,1798,1770,Alabama State University,Montgomery,AL,36104-0271,9,2,-86.295677,32.364317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,496681,2021,90,18,72,Florida Academy of Nursing,Miramar,FL,33025,5,1,-80.306128,25.978064
3752,496991,2021,63,0,63,Galen Health Institutes-Miami Campus,Pembroke Pines,FL,33027,5,1,-80.337924,25.997073
3753,497046,2021,276,136,140,Madera Community College,Madera,CA,93638,4,2,-119.997870,36.925132
3754,497277,2021,18,2,16,Arizona College-Glendale,Glendale,AZ,85302,3,1,-112.154175,33.566263


In [155]:
#Dropping duplicates
dirty_test.drop_duplicates()

,UNITID,Year,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,100654,2019,4036,2085,1951,Alabama A & M University,Normal,AL,35762,9,3,-86.568502,34.783368
1,100663,2019,45196,25012,20184,University of Alabama at Birmingham,Birmingham,AL,35294-0110,9,5,-86.799345,33.505697
2,100690,2019,693,553,140,Amridge University,Montgomery,AL,36117-3553,9,1,-86.174010,32.362609
3,100706,2019,6833,3288,3545,University of Alabama in Huntsville,Huntsville,AL,35899,9,3,-86.640449,34.724557
4,100724,2019,3568,1798,1770,Alabama State University,Montgomery,AL,36104-0271,9,2,-86.295677,32.364317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,496681,2021,90,18,72,Florida Academy of Nursing,Miramar,FL,33025,5,1,-80.306128,25.978064
3752,496991,2021,63,0,63,Galen Health Institutes-Miami Campus,Pembroke Pines,FL,33027,5,1,-80.337924,25.997073
3753,497046,2021,276,136,140,Madera Community College,Madera,CA,93638,4,2,-119.997870,36.925132
3754,497277,2021,18,2,16,Arizona College-Glendale,Glendale,AZ,85302,3,1,-112.154175,33.566263


In [156]:
#Adding df for string values
higher_ed_names = dirty_test[['INSTNM','CITY', 'STABBR','ZIP']]
higher_ed_names

,INSTNM,CITY,STABBR,ZIP
0,Alabama A & M University,Normal,AL,35762
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110
2,Amridge University,Montgomery,AL,36117-3553
3,University of Alabama in Huntsville,Huntsville,AL,35899
4,Alabama State University,Montgomery,AL,36104-0271
...,...,...,...,...
3751,Florida Academy of Nursing,Miramar,FL,33025
3752,Galen Health Institutes-Miami Campus,Pembroke Pines,FL,33027
3753,Madera Community College,Madera,CA,93638
3754,Arizona College-Glendale,Glendale,AZ,85302


In [157]:
#Dropping string values
clean_test = dirty_test.drop(['INSTNM', 'CITY', 'STABBR','ZIP'], axis=1)
clean_test.sample(10)

,UNITID,Year,HRTOTLT,HRTOTLM,HRTOTLW,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
433,127389,2019,367,182,185,3,1,-102.616144,38.068886
1752,192509,2019,405,108,297,4,1,-73.829457,40.760109
2840,236887,2020,1878,1077,801,5,2,-118.276505,46.078557
603,138956,2019,2660,1162,1498,4,2,-82.048439,33.418992
403,126012,2019,612,139,473,9,1,-122.253168,37.868221
2952,241331,2020,850,369,481,9,2,-66.113908,18.466573
458,129020,2019,36551,20100,16451,9,5,-72.249948,41.809098
3071,368443,2020,289,94,195,5,1,-87.312856,36.582865
2581,225070,2019,2363,881,1482,5,2,-96.633891,33.705740
3167,417327,2020,81,27,54,3,1,-81.402219,28.663604


In [158]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(clean_test, columns=['HLOFFER','INSTSIZE'])
X

,UNITID,Year,HRTOTLT,HRTOTLM,HRTOTLW,LONGITUD,LATITUDE,HLOFFER_3,HLOFFER_4,HLOFFER_5,HLOFFER_6,HLOFFER_7,HLOFFER_8,HLOFFER_9,INSTSIZE_-2,INSTSIZE_1,INSTSIZE_2,INSTSIZE_3,INSTSIZE_4,INSTSIZE_5
0,100654,2019,4036,2085,1951,-86.568502,34.783368,0,0,0,0,0,0,1,0,0,0,1,0,0
1,100663,2019,45196,25012,20184,-86.799345,33.505697,0,0,0,0,0,0,1,0,0,0,0,0,1
2,100690,2019,693,553,140,-86.174010,32.362609,0,0,0,0,0,0,1,0,1,0,0,0,0
3,100706,2019,6833,3288,3545,-86.640449,34.724557,0,0,0,0,0,0,1,0,0,0,1,0,0
4,100724,2019,3568,1798,1770,-86.295677,32.364317,0,0,0,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,496681,2021,90,18,72,-80.306128,25.978064,0,0,1,0,0,0,0,0,1,0,0,0,0
3752,496991,2021,63,0,63,-80.337924,25.997073,0,0,1,0,0,0,0,0,1,0,0,0,0
3753,497046,2021,276,136,140,-119.997870,36.925132,0,1,0,0,0,0,0,0,0,1,0,0,0
3754,497277,2021,18,2,16,-112.154175,33.566263,1,0,0,0,0,0,0,0,1,0,0,0,0


In [159]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
X_scaled

array([[-1.14456595, -0.56417646,  0.05276353, ...,  2.60315146,
        -0.31395355, -0.2476222 ],
       [-1.14448616, -0.56417646,  4.88528091, ..., -0.38414976,
        -0.31395355,  4.03841005],
       [-1.1442468 , -0.56417646, -0.33973175, ..., -0.38414976,
        -0.31395355, -0.2476222 ],
       ...,
       [ 2.36954778,  3.59489478, -0.38869092, ..., -0.38414976,
        -0.31395355, -0.2476222 ],
       [ 2.37159566,  3.59489478, -0.41898221, ..., -0.38414976,
        -0.31395355, -0.2476222 ],
       [ 2.37167544,  3.59489478, -0.41992148, ..., -0.38414976,
        -0.31395355, -0.2476222 ]])

In [160]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

higher_ed_pca = pca.fit_transform(X_scaled)
higher_ed_pca

array([[ 1.04395578, -1.04075903, -1.65074808],
       [ 9.18275021,  1.95469946,  0.89565641],
       [-0.07811246,  0.14218996, -0.14777016],
       ...,
       [-1.60206177,  1.73467539,  0.46302141],
       [-2.21709126,  3.95178593, -0.85080607],
       [-2.00153067,  3.48996446, -0.98287284]])

In [161]:
# Create a DataFrame with the 3 principal components.
pca_df = pd.DataFrame(data=higher_ed_pca, columns=['Prin_Comp 1','Prin_Comp 2','Prin_Comp 3'], index=X.index)
pca_df.sample(10)

,Prin_Comp 1,Prin_Comp 2,Prin_Comp 3
1203,-0.567812,-1.594127,-0.209812
1952,2.503176,0.037911,-0.556944
318,1.641453,-0.833744,-2.041857
3522,-1.806041,2.667634,-0.762594
181,4.595622,0.703273,0.021979
3649,-0.707896,1.008828,0.855382
587,-0.612290,-1.475013,1.871252
1914,0.468843,-1.210294,0.845381
194,-0.297936,-1.012111,-3.144312
951,-0.224949,-0.904309,-2.868506


In [162]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1,11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)
    
elbow_data = {'k' : k, 'inertia' : inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.hvplot.line(x='k', y='inertia', title='Elbow Curve', xticks=k)

:Curve   [k]   (inertia)

In [163]:
# Initialize the K-Means model.
model = KMeans(n_clusters=5, random_state=1)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)
predictions

array([0, 2, 1, ..., 3, 3, 3])

In [164]:
# Create a new DataFrame including predicted clusters.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([clean_test,pca_df], axis= 1, join="inner")

#  Add a new column. 
clustered_df = pd.concat([clustered_df,higher_ed_names], axis= 1, join="inner")

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.sample(10)

(3756, 17)


,UNITID,Year,HRTOTLT,HRTOTLM,HRTOTLW,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,Prin_Comp 1,Prin_Comp 2,Prin_Comp 3,INSTNM,CITY,STABBR,ZIP,Class
1136,164155,2019,5335,3744,1591,5,2,-76.481692,38.980507,0.133252,-1.065058,1.499745,United States Naval Academy,Annapolis,MD,21402-5000,1
2797,234669,2020,3034,1313,1721,5,4,-122.150118,47.584014,-0.066157,0.696921,-0.590581,Bellevue College,Bellevue,WA,98007,0
1830,196088,2019,30029,17759,12270,9,5,-78.789458,43.000942,6.583924,1.305183,0.717861,University at Buffalo,Buffalo,NY,14260-1660,4
3674,488387,2021,9,0,9,7,1,-117.716117,34.094273,-2.159274,3.639561,1.149799,Claremont Lincoln University,Claremont,CA,91711,3
1153,164872,2019,454,293,161,7,1,-71.085786,42.348467,-0.945529,-0.115628,1.031634,Boston Architectural College,Boston,MA,02115-2703,1
2569,224527,2019,1685,1023,662,7,2,-94.374488,32.557847,-0.550679,-1.196764,1.752659,East Texas Baptist University,Marshall,TX,75670-1498,1
1575,183275,2019,129,115,14,5,1,-71.484370,42.806830,-1.057036,0.400964,0.600645,Thomas More College of Liberal Arts,Merrimack,NH,3054,1
3608,483975,2020,846,522,324,9,1,-85.340805,31.206294,-0.926841,2.861803,0.384378,Alabama College of Osteopathic Medicine,Dothan,AL,36303,3
2521,221661,2019,2904,1351,1553,9,2,-85.052088,35.048254,0.474332,-1.086406,0.828465,Southern Adventist University,Collegedale,TN,37315-0370,1
2286,213303,2019,707,256,451,7,2,-75.777456,41.558969,-0.636703,-1.626512,1.872190,Keystone College,La Plume,PA,18440-0200,1


In [165]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x='Prin_Comp 1',
    y='Prin_Comp 2',
    z='Prin_Comp 3',
    color='Class',
    symbol='Class',
    hover_name= 'INSTNM',
    hover_data= ['HLOFFER'],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [166]:
#Creating table with hvplot. 
clustered_df.hvplot.table(
    columns=['UNITID','HRTOTLT','HRTOTLM','HRTOTLW','INSTNM'],
    sortable=True,
    selectable=True
)

:Table   [UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM]

In [167]:
#Printing the total number of schools.
print(f'There are {len(clustered_df)} schools.')

There are 3756 schools.


In [168]:
#Scaling data to create the scatter plot with schools. 
clustered_scaled = MinMaxScaler().fit_transform(clustered_df[['HRTOTLM', 'HRTOTLW']])
clustered_scaled

array([[2.52601737e-02, 3.13670638e-02],
       [3.03025163e-01, 3.24506825e-01],
       [6.69970075e-03, 2.25084005e-03],
       ...,
       [1.64766601e-03, 2.25084005e-03],
       [2.42303825e-05, 2.57238862e-04],
       [1.21151912e-04, 0.00000000e+00]])

In [169]:
#Create a new datafram that has the scaled data with the clustered dataframe index....
plot_df = pd.DataFrame(
    data=clustered_scaled, columns=["HRTOTLM", "HRTOTLW"], index=clustered_df.index.copy()
)

# Add the school name column from the clustered_df DataFrame to the new DataFrame.
plot_df['INSTNM'] = clustered_df['INSTNM']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df['Class']

plot_df.head(10)

,HRTOTLM,HRTOTLW,INSTNM,Class
0,0.025260,0.031367,Alabama A & M University,0
1,0.303025,0.324507,University of Alabama at Birmingham,2
2,0.006700,0.002251,Amridge University,1
3,0.039835,0.056994,University of Alabama in Huntsville,0
4,0.021783,0.028457,Alabama State University,1
5,0.166354,0.200598,The University of Alabama,4
6,0.003635,0.005354,Central Alabama Community College,1
7,0.007330,0.011045,Athens State University,1
8,0.021698,0.029116,Auburn University at Montgomery,0
9,0.163834,0.162848,Auburn University,4


In [170]:
#Creating a hvplot.scatter.
plot_df.hvplot.scatter(
    x="HRTOTLM", 
    y="HRTOTLW",
    by="Class",
    hover_cols="INSTNM"
)

:NdOverlay   [Class]
   :Scatter   [HRTOTLM]   (HRTOTLW,INSTNM)

In [171]:
clustered_df.sample(10)

,UNITID,Year,HRTOTLT,HRTOTLM,HRTOTLW,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,Prin_Comp 1,Prin_Comp 2,Prin_Comp 3,INSTNM,CITY,STABBR,ZIP,Class
511,132657,2019,2937,1705,1232,9,2,-80.122806,26.385985,0.546572,-1.185468,0.902057,Lynn University,Boca Raton,FL,33431-5598,1
1149,164720,2019,839,375,464,7,2,-71.811894,42.269171,-0.541220,-1.861622,1.876424,Becker College,Worcester,MA,01609-2165,1
2416,217925,2019,1100,786,314,9,2,-81.074430,34.075510,0.168387,-1.163200,0.854305,Columbia International University,Columbia,SC,29203-1513,1
3130,403469,2020,97,30,67,3,2,-88.392194,38.380271,-1.388425,0.570597,-0.008759,Frontier Community College,Fairfield,IL,62837,3
1348,174020,2019,2336,969,1367,9,3,-93.074361,44.957256,0.726793,-1.138761,-1.801969,Metropolitan State University,Saint Paul,MN,55106-5000,0
2642,228158,2019,3902,1871,2031,3,3,-102.365075,33.575904,-0.007948,-0.326293,-2.780908,South Plains College,Levelland,TX,79336,0
3003,243601,2020,4081,1878,2203,9,4,-66.006259,18.239647,0.728784,1.391923,-0.165971,Universidad Ana G. Mendez-Gurabo Campus,Gurabo,PR,778,0
2861,237640,2020,208,98,110,7,1,-81.537808,39.312447,-1.421647,1.317397,1.166664,Ohio Valley University,Vienna,WV,26105,3
296,119164,2019,6619,3075,3544,3,5,-117.846629,34.047764,1.963985,0.321469,-1.223044,Mt San Antonio College,Walnut,CA,91789-1399,0
755,146667,2019,485,371,114,9,1,-89.346721,40.148924,-0.126998,0.072335,-0.229683,Lincoln Christian University,Lincoln,IL,62656-2111,1
